In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
cg2021 = pd.read_spss("2021/cgnac2021.sav", usecols=['Identificador', 'cg_vive_predio_pr', 'cg_dista_pre_viv'])
eu2021 = pd.read_spss("2021/eunac2021.sav", usecols=['Identificador', 'eu_k1301', 'eu_k1304', 'eu_k1307', 'fact_exp_fin']).fillna(0)

#### Selección de casos productivos 

In [3]:
eu2021 = eu2021.loc[eu2021.fact_exp_fin != 0]

#### Unión de bases: Empleo + Características Generales

In [4]:
eucg = pd.merge(eu2021, cg2021, on = 'Identificador')

## Variables de tiempo dedicado al trabajo

Esta información no se encuentra publicada en la página del INEC, por lo que fué necesario solicitar dicha información. La base que contiene las variables de tiempo dedicado a la labor agropecuaria es: `eunac2021_nueva.sav`

In [5]:
empleo2021 = pd.read_spss("2021/eunac2021_nueva.sav").fillna(0)

In [6]:
#Identificador
empleo2021['Identificador'] = empleo2021.ual_prov + empleo2021.ual_cant + empleo2021.ual_parr + \
                              empleo2021.ual_estr + empleo2021.ual_segm + empleo2021.al_ncues

#### Tiempo promedio trabajado por persona
- Semanas promedio trabajadas por trimestre
- Horas promedio trabajadas a la semana

In [7]:
#Persona productora
pp_sem = ['tfsr_prosempp_pt', 'tfsr_prosempp_st', 'tfsr_prosempp_tt', 'tfsr_prosempp_ct']
pp_hrs = ['tfsr_prohorpp_pt', 'tfsr_prohorpp_st', 'tfsr_prohorpp_tt', 'tfsr_prohorpp_ct']

#Trabajadores familiares
tf_sem = ['tfsr_prosemt_pt', 'tfsr_prosemt_st', 'tfsr_prosemt_tt', 'tfsr_prosemt_ct']
tf_hrs = ['tfsr_prohort_pt', 'tfsr_prohort_st', 'tfsr_prohort_tt', 'tfsr_prohort_ct']

#### Número de trabajadores por género y tipo

In [8]:
pp_h = ['tfsr_trafapph_pt', 'tfsr_trafapph_st', 'tfsr_trafapph_tt', 'tfsr_trafapph_ct']
pp_m = ['tfsr_trafappm_pt', 'tfsr_trafappm_st', 'tfsr_trafappm_tt', 'tfsr_trafappm_ct']

tf_h = ['tfsr_trafamh_pt', 'tfsr_trafamh_st', 'tfsr_trafamh_tt', 'tfsr_trafamh_ct']
tf_m = ['tfsr_trafamm_pt', 'tfsr_trafamm_st', 'tfsr_trafamm_tt', 'tfsr_trafamm_ct']

#### Horas totales trabajadas por trimestre

In [9]:
#Persona productora
pp = np.add(empleo2021[pp_h],empleo2021[pp_m])
pp_hrs_trim_prom = np.multiply(empleo2021[pp_sem], empleo2021[pp_hrs])
pp_hrs_trim_totl = np.multiply(pp_hrs_trim_prom, pp)

#Trabajadores familiares
tf = np.add(empleo2021[tf_h],empleo2021[tf_m])
tf_hrs_trim_prom = np.multiply(empleo2021[tf_sem], empleo2021[tf_hrs])
tf_hrs_trim_totl = np.multiply(tf_hrs_trim_prom, tf)

#### Horas totales trabajadas por año (persona productora y trabajadores familiares)

In [10]:
ppf = pp_hrs_trim_totl.sum(axis=1) + tf_hrs_trim_totl.sum(axis=1)

In [11]:
empleo2021['hrs_tot'] = ppf                             # Añadiendo a la nueva base el campo de horas totales trabajadas
empleo_hrs = empleo2021.loc[:,'Identificador':]         # Seleccionando el campo Identificador y hrs_tot

#### Unión de bases: (Empleo + Características generales) + Horas empleadas

In [12]:
consolidado = pd.merge(eucg, empleo_hrs, on='Identificador', how='left')

## Criterios AFC
#### Criterio residencia

In [13]:
c1 = consolidado['cg_vive_predio_pr'] == 'SI'
c2 = consolidado['cg_dista_pre_viv'].between(0, 50)

f"El {consolidado[c1|c2].shape[0] / consolidado.shape[0]:.2%} de los productores cumple con el criterio de residencia"

'El 90.62% de los productores cumple con el criterio de residencia'

#### Criterio mano de obra familiar

In [14]:
consolidado['manodeobra'] = consolidado['eu_k1304'] / (consolidado['eu_k1304'] + consolidado['eu_k1307'])
c3 = consolidado['manodeobra'] >= 0.6

'El {:.2%} de los productores cumple con el criterio de mano de obra'.format(consolidado[c3].shape[0] / consolidado.shape[0])

'El 85.81% de los productores cumple con el criterio de mano de obra'

#### Criterio tiempo completo

In [15]:
c4 = consolidado['hrs_tot'] >= 1920

'El {:.2%} de los productores cumple con el criterio de tiempo completo'.format(consolidado[c4].shape[0] / consolidado.shape[0])

'El 52.10% de los productores cumple con el criterio de tiempo completo'

#### Todos los criterios

In [16]:
'El {:.2%} de los productores cumple con todos los criterios'.format(consolidado[(c1|c2) & c3 & c4].shape[0] / consolidado.shape[0])

'El 45.97% de los productores cumple con todos los criterios'

In [17]:
#consolidado[(c1|c2) & c3 & c4]